# Homework 01 - Point Operations and Histograms

Contact: David C. Schedl (david.schedl@fh-hagenberg.at)

Note: this is the starter pack for the **Digital Imaging / Computer Vision** homework. Note: You do not need to use the exact same template and can start from scratch as well!

## Content

- [Task A: Automatic contrast adjustment ](#Task-A)
- [Task B: Histogram Matching ](#Task-B)

# Setup

Let's import useful libraries, first. 
We'll download test images (`cat.jpg` and `gogh.jpg`) from the internet. 
Then let's define a function to display images and their histograms.

In [ ]:
import os
import cv2 # openCV
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.express.colors import sample_colorscale
from plotly.subplots import make_subplots

!curl -o "cat.jpg" "http://placekitten.com/367/480" --silent
!curl -o "gogh.jpg" "https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Vincent_van_Gogh_-_National_Gallery_of_Art.JPG/367px-Vincent_van_Gogh_-_National_Gallery_of_Art.JPG" --silent


def show_hist_stats(image: np.ndarray, use_cumulative: bool = False):
    """" Function to create a histogram of an image and optionally display the stats
    
    Args:
        image: The image to create the histogram for
        use_cumulative: Whether to use the cumulative histogram or not

    Returns:
        The figure object
    """
    
    x = np.linspace(0, 1, 5)
    c = sample_colorscale('HSV', list(x))

    # 8-bit (256) image histogram
    counts, bins = np.histogram(image.ravel(), bins=range(257))
    cumulative = np.cumsum(counts)

    fig = px.bar(x=bins[:-1], y=cumulative if use_cumulative else counts, labels={'x':'pixel value', 'y':'count'}, color_discrete_sequence=['black']*256)

    fig.update_layout(plot_bgcolor='white', margin=dict(t=0, b=0, r=0, l=0, pad=0))

    num_markers = 1000
    y_pos = -np.max(cumulative if use_cumulative else counts)*.05
    fig.add_traces([
        go.Scatter(x=np.linspace(0,255,num_markers), y=[y_pos]*num_markers, mode='markers', marker={'color': np.linspace(0,255,num_markers), 'colorscale': 'gray', 'size': 10, 'symbol': 'square' }),
    ])


    # show the mean, median, mode and std as vertical lines
    mean_value = np.mean(image)
    median_value = np.median(image)
    std_value = np.std(image)
    mode_value = np.argmax(counts)
    min_value = np.min(image)
    max_value = np.max(image)

    return fig

def show_image_and_hist(img, use_cumulative=False):
    """ Function to display an image and its histogram side by side

    Args:
        img: The image to display
        use_cumulative: Whether to use the cumulative histogram or not

    Returns:
        The figure object
    """
    fig = make_subplots(1, 2)
    fig.add_trace(go.Image(z=cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), name="Image"), 1, 1)
    traces = show_hist_stats(img, use_cumulative=use_cumulative).data
    for trace in traces:
        fig.add_trace(trace, 1, 2)
    fig.show()


grayscale = cv2.imread("gogh.jpg", cv2.IMREAD_GRAYSCALE)
show_image_and_hist(grayscale, use_cumulative=False)


## Task A
<a name="Task-A" id="Task-A"> </a>

Implement an automatic contrast adjustment algorithm for an 8-bit grayscale image.

The contrast adjustment is a linear point operation ($ f(a) = k a + d $) that scales and offsets pixels ($a$) in the input image such that the contrast is increased. 
The scaling factor $k$ is computed as the ratio between the new maximum and minimum pixel value, ${a_{hi}, a_{lo}}$, in the input image:
$$ k = \frac{255}{a_{hi} - a_{lo}}. $$
The offset is the difference between the minimum pixel value and 0 (the new minimum): $$d = -a_{lo}.$$
The result of this operation is that the minimum pixel value ($a_{lo}$) in the input image is mapped to 0 and the maximum pixel value ($a_{hi}$) is mapped to 255. <br>
Typically, the minimum and maximum pixel values (${a_{hi}, a_{lo}}$) are chosen such that a certain percentage of pixels is darker and brighter. Darker and brighter pixels are clipped to 0 and 255, respectively.
A typical choice is to clip the 1% darkest and 1% brightest pixels.

Implement an auto-contrast function that takes an image and the percentage of pixels to be darker and brighter as input and returns the contrast-adjusted image.
A possible signature of the function is:
```python
def auto_contrast(img, hi_lo = 0.01):
    # ...
    return modified_img
```

**Hint:** You can use an image histogram or a cumulated histogram to find the values for $a_{hi}$ and $a_{lo}$ given a percentage.


In [ ]:
# Solution Task A

def auto_contrast(img, hi_lo = 0.01):
    """ Function to perform auto contrast on an image

    Args:
        img: The image to perform auto contrast on
        hi_lo: The percentage of pixels to clip from the top and bottom

    Returns:
        The image with auto contrast applied
    """
    # Todo: Implement auto contrast
    modified_img = img.copy()
    return modified_img

grayscale = cv2.imread("gogh.jpg", cv2.IMREAD_GRAYSCALE)
# test the auto contrast function
auto_contrast_img = auto_contrast(grayscale, hi_lo = .01)
show_image_and_hist(auto_contrast_img, use_cumulative=False)

## Task B
<a name="Task-B" id="Task-B"> </a>

Implement histogram matching for 8-bit grayscale images. 
The algorithm should take two images as input and return the first image modified such that its histogram matches the histogram of the second image. <br>
Do not use any built-in histogram matching functions but implement the algorithm yourself.
See the lecture slides for details and use the `skimage.exposure.match_histogram` function to evaluate your solution. Your solution does not need to yield exactly the same result as the skimage function, but should be close.

A possible signature of the function is:
```python
def match_histograms(img, ref):
    # ...
    return matched_img
```

**Hint(s):** 
You can use the cumulated histograms for computing a mapping.  

The mapping (in a variable `mapping`) can be applied as in our Tutorial:
```python
def histMatch(a):
    return mapping[a]

matched = np.vectorize(histMatch)(img)
```
Furthermore, make sure the images have the same resolution and that the pixel values are in the range [0, 255].


In [ ]:
# Solution Task B

def match_histograms(img, ref):
    """ Function to perform histogram matching on an image

    Args:
        img: The image to perform histogram matching on
        ref_img: The reference image to match the histogram to

    Returns:
        The image with histogram matching applied
    """

    # check that the images are the same size
    assert img.shape == ref.shape, "The images must be the same size"

    HA = np.cumsum(np.histogram(img.ravel(), bins=range(257))[0]).astype(np.float32)
    HR = np.cumsum(np.histogram(ref.ravel(), bins=range(257))[0]).astype(np.float32)
    # (optionally) normalize the cumulated histograms
    HA = HA / HA[-1]
    HR = HR / HR[-1]

    # Todo: Implement histogram matching
    
    matched = img.copy()
    return matched


# ---------------------------------------------
# test your implementation
reference = cv2.imread("gogh.jpg", cv2.IMREAD_GRAYSCALE)
image = cv2.imread("cat.jpg", cv2.IMREAD_GRAYSCALE)

# using skimage's match_histograms
matched = match_histograms(image, reference).astype(np.uint8)

# display images
imgs = [reference, image, matched]
titles = ['Reference', 'Image', 'Matched']

fig = make_subplots(2, len(imgs), subplot_titles=titles,
    horizontal_spacing = 0.05, vertical_spacing = 0.1)
for i, (img, title) in enumerate(zip(imgs, titles)):
    fig.add_trace(go.Image(z=cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), name="Image"), 1, i+1)
    traces = show_hist_stats(img, use_cumulative=True).data
    for trace in traces:
        fig.add_trace(trace, 2, i+1)

fig.show()

### Comparison to `skimage.exposure.match_histogram`

Below you find an implementation with `skimage.exposure.match_histogram` to evaluate your solution. 

In [ ]:
import skimage.exposure as ske

reference = cv2.imread("gogh.jpg", cv2.IMREAD_GRAYSCALE)
image = cv2.imread("cat.jpg", cv2.IMREAD_GRAYSCALE)

# using skimage's match_histograms
matched = ske.match_histograms(image, reference).astype(np.uint8)

# display images
imgs = [reference, image, matched]
titles = ['Reference', 'Image', 'Matched']

fig = make_subplots(2, len(imgs), subplot_titles=titles,
    horizontal_spacing = 0.05, vertical_spacing = 0.1)
for i, (img, title) in enumerate(zip(imgs, titles)):
    fig.add_trace(go.Image(z=cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), name="Image"), 1, i+1)
    traces = show_hist_stats(img, use_cumulative=True).data
    for trace in traces:
        fig.add_trace(trace, 2, i+1)

fig.show()

## Further comments/hints:
*   You do not need to come up with a super efficient implementations! It is mostly about getting into the topic.
*   Think about the problem, solve it, and evaluate your solutions on a few test images (you can pick them yourself).
*   Summarize your ideas and solutions in the report. 


**Have fun!** 😸
